In [12]:
import os
from tqdm  import tqdm
import pandas as pd

## 1. 함수 정의

In [32]:
from PIL import Image
import json
# -----------Properties-----------
#이미지 shape 가져오는 함수
def get_image_shape(path):
    image_input = Image.open(path)
    try:
        return (image_input.size[0], image_input.size[1], 3)
    except AttributeError:
        print('error : ', path)
        return (None, None, None)
    
#properties의 coco format을 yolo format으로 변환시켜주는 함수
def coco_to_yolo(img_path, x, y, w, h):
    image_shape = get_image_shape(img_path)
    x_center = (x + (w/2)) / image_shape[1]
    y_center = (y + (h/2)) / image_shape[0]
    width = w / image_shape[1]
    height = h / image_shape[0]
    return x_center, y_center, width, height

#properties 파일일 때 annotation 작성하는 함수
def if_properties(filepath, class_number, food_name):
    with open(filepath + 'crop_area.properties', 'r') as f:
        content = f.read()
        #모두 맨 뒤에는 [''] 존재하므로 [:-1]으로 지정
        content_list = content.split('\n')[:-1]
        for i in content_list:
            image_dir = filepath.split('label')[0]
            image_name = i.split('=')[0]
            img_path = f'{image_dir}image/{image_name}.jpg'
            if len(i.split('=')[1].split(',')) <= 3:
                print(i)
            x, y, w, h = i.split('=')[1].split(',')
            x_center, y_center, width, height = coco_to_yolo(img_path, int(x), int(y), int(w), int(h))
            label_dir = f'./labels/{food_name}/'
            if not os.path.exists(label_dir):
                os.makedirs(label_dir)
            with open(label_dir + f'{image_name}.txt', 'w') as ff:
                ff.write(f'{class_number} {x_center} {y_center} {width} {height} ')
                
# -----------JSON-----------
#json 파일일 때 annotation 작성하는 함수
def if_json(filepath, class_number, food_name):
    with open(filepath, 'r') as f:
        file = json.load(f)[0]
        img_name = file['Code Name'][:-4]
        x_center, y_center = list(map(float, file['Point(x,y)'].split(',')))
        width, height = float(file['W']), float(file['H'])
    label_dir = f'./labels/{food_name}/'
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
    with open(label_dir + f'{img_name}.txt', 'w') as ff:
        ff.write(f'{class_number} {x_center} {y_center} {width} {height} ')

## 2. Run

In [33]:
%%time
food_list = os.listdir('./kfood_sy')
class_df = pd.read_csv('./class_df.csv')

for i in tqdm(food_list):
    label_path = f'./kfood_sy/{i}/label/'
    for file in os.listdir(label_path):
        file_path = label_path + file
        if file.split('.')[-1] == 'properties':
            if_properties(label_path, class_df[class_df['name'] == i]['class'].iloc[0], i)
        elif file.split('.')[-1] == 'json':
            if_json(label_path + file, class_df[class_df['name'] == i]['class'].iloc[0], i)
        else:
            print('error!!!!!!', file_path)

100%|██████████| 82/82 [00:25<00:00,  3.17it/s]

CPU times: user 21.2 s, sys: 4.6 s, total: 25.8 s
Wall time: 25.8 s


## 3. 확인

In [28]:
#전체 파일 개수 세기
count_dic = {}
for i in food_list:
    count_dic[i] = len(os.listdir(f'./kfood_sy/{i}/image'))
sum(count_dic.values())

85583

In [34]:
#annotation 완료한 파일 개수 세기
label_count_dic = {}
for i in food_list:
    label_count_dic[i] = len(os.listdir(f'./labels/{i}'))
sum(label_count_dic.values())

85583

In [40]:
for i in food_list:
    print(i, count_dic[i] - label_count_dic[i])

된장찌개 0
미역국 0
새우볶음밥 0
고추장진미채볶음 0
주먹밥 0
잡곡밥 0
치즈피자 0
떡볶이 0
갈비찜 0
우동 0
유부초밥 0
알밥 0
감자튀김 0
파전 0
콤비네이션피자 0
달걀후라이 0
짬뽕 0
해장국 0
비빔냉면 0
감자채볶음 0
게장 0
후라이드치킨 0
양념치킨 0
쪽갈비구이 0
삼계탕 0
냉면 0
닭갈비 0
마늘장아찌 0
비빔밥 0
만두국 0
비빔국수 0
부대찌개 0
보쌈 0
모듬튀김 0
장어구이 0
찐만두 0
새우튀김 0
쭈꾸미볶음 0
시금치나물 0
달걀말이 0
갈비구이 0
제육볶음 0
김치볶음밥 0
깍두기 0
잡채 0
송편 0
고등어구이 0
닭찜 0
불고기 0
육회 0
두부김치 0
훈제오리 0
오므라이스 0
순대 0
경단 0
소세지볶음 0
가지구이 0
족발 0
삼겹살구이 0
떡국 0
생선전 0
떡갈비 0
건새우볶음 0
장조림 0
숙주나물 0
한과 0
김치찌개 0
애호박볶음 0
마르게리타피자 0
짜장면 0
해물찜 0
고추장아찌 0
고르곤졸라피자 0
라면 0
군만두 0
회무침 0
콩국수 0
김밥 0
배추김치 0
카레라이스 0
흰죽 0
곱창전골 0


In [5]:
import os
from tqdm.notebook import tqdm

for folder in ['train', 'test', 'val']:
    n = len(os.listdir(f'./datasets/kfood/labels/{folder}'))
    print('################',folder, '################')
    for i in tqdm(range(n)):
        file_dir = f'./datasets/kfood/labels/{folder}/'
        filename = os.listdir(file_dir)[i]
        # print(filename)
        if filename[0] == 'I':
            # print(filename)
            with open(file_dir + filename, 'r') as file:
                file = file.read().split()
                # print(file)
            with open(file_dir + filename, 'w') as modify:
                file[1] = min(1.0, float(file[1]))
                file[2] = min(1.0, float(file[2]))
                file[3] = min(1.0, float(file[3]))
                file[4] = min(1.0, float(file[4]))
        
                modify.write(f'{file[0]} {file[1]} {file[2]} {file[3]} {file[4]}')

################ train ################


  0%|          | 0/68457 [00:00<?, ?it/s]

################ test ################


  0%|          | 0/8568 [00:00<?, ?it/s]

################ val ################


  0%|          | 0/8557 [00:00<?, ?it/s]

'Img_080_0524.txt'